In [1]:
import docx
import re
import pandas as pd

import os
os.chdir("../")


In [4]:
from src.utils import extract_qa_pairs_to_df

file_path = "data/evaluation_sets/Robinson Q&A.docx"  # Replace with your actual file path
df = extract_qa_pairs_to_df(file_path)

In [6]:
df.head()

,question,ground_truths
0,Who are the parties to the Agreement and what ...,Cloud Investments Ltd. (“Company”) and Jack Ro...
1,What is the termination notice?,According to section 4:14 days for convenience...
2,What are the payments to the Advisor under the...,According to section 6: 1. Fees of $9 per hour...
3,Can the Agreement or any of its obligations be...,1. Under section 1.1 the Advisor can’t assign ...
4,Who owns the IP?,According to section 4 of the Undertaking (App...


#### Upload the dataset to langsmith

In [8]:
# from ragas.integrations.langsmith import upload_dataset

# dataset_name = "Raptor Q&A"
# dataset_desc = "Evaluation set for Question and Answer contract advisor"

# dataset = upload_dataset(df, dataset_name, dataset_desc)

In [ ]:
how can I 

#### Evaluation with Ragas

In [10]:
from ragas.integrations.langchain import EvaluatorChain
from ragas.integrations.langsmith import evaluate

# the metric we will be using
from ragas.metrics import answer_correctness

In [ ]:
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


generator = prompt | llm | StrOutputParser()

retriever = RunnableParallel(
    {
        "context": vectorstore_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
)

filter_langsmith_dataset = RunnableLambda(
    lambda x: x["question"] if isinstance(x, dict) else x
)

rag_chain = RunnableParallel(
    {
        "question": filter_langsmith_dataset,
        "answer": filter_langsmith_dataset | retriever | generator,
        "contexts": filter_langsmith_dataset
        | vectorstore_retriever
        | ragas_output_parser,
    }
)

In [ ]:
# Evaluate Rag chain first
dataset_name = "Mere Christianity"
# evaluate just llms
run = evaluate(
    dataset_name=dataset_name,
    llm_or_chain_factory=rag_chain,
    experiment_name="rag_chain_1",
    metrics=[answer_correctness],
    verbose=True,
)